<a href="https://colab.research.google.com/github/JaniceSilva/arboviroses-platform/blob/master/train_lstm_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treino LSTM por cidade + Push para GitHub
Este notebook deve ser aberto no Google Colab.

Para usar este notebook, você precisará adicionar seu token do GitHub como um segredo do Colab com o nome `GITHUB_TOKEN`.

In [ ]:
GITHUB_USER = "JaniceSilva"
REPO_NAME = "arboviroses-platform"
# Para usar o token do GitHub, adicione-o aos segredos do Colab com o nome 'GITHUB_TOKEN'.# Para saber mais, consulte: https://colab.research.google.com/notebooks/basic_features_overview.ipynb#scrollTo=svGE4fLzmtXLfrom google.colab import userdata
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')